# Final Project - Shawn Feng, Will Novak, Eddie Loyd

In [12]:
import collections
import datetime
import fluidsynth
import glob
import numpy as np
import pathlib
import pandas as pd
import pretty_midi
import seaborn as sns
import tensorflow as tf

from IPython import display
from matplotlib import pyplot as plt
from typing import Optional

_SAMPLING_RATE = 16000

In [13]:
filenames = glob.glob('midi_data/arpeggios/**/*.mid', recursive=True)
sample_file = filenames[0]  # Change this to the desired MIDI file
sample_file

'midi_data/arpeggios\\aug\\A-1-aug-arpeggio-0.mid'

In [14]:
pm = pretty_midi.PrettyMIDI(sample_file)
print(pm)

In [15]:
def display_audio(pm: pretty_midi.PrettyMIDI, seconds=30):
    waveform = pm.fluidsynth(fs=_SAMPLING_RATE)
    # Take a sample of the generated waveform to mitigate kernel resets
    waveform_short = waveform[:seconds*_SAMPLING_RATE]
    return display.Audio(waveform_short, rate=_SAMPLING_RATE)

In [16]:
display_audio(pm)

In [17]:
# Create a FluidSynth synthesizer from pretty_midi
fs = pm.fluidsynth(fs=_SAMPLING_RATE)

fs

array([0.        , 0.00113636, 0.        , ..., 0.        , 0.00113636,
       0.00113636])

In [19]:
# Generate the waveform
waveform = fs.get_waveform()

# Display the audio
display(Audio(pm.fluidsynth(fs=_SAMPLING_RATE), rate=_SAMPLING_RATE))

TypeError: 'module' object is not callable